In [1]:
from osgeo import gdal
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
fast=cv2.FastFeatureDetector_create()
sift=cv2.SIFT_create()
bf=cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

In [3]:
com_name='AHI8_OBI_2000M_NOM_'
time_point='20200901'
name_list=['_0000','_0020','_0040']

In [4]:

raster=gdal.Open(com_name+time_point+name_list[0]+'.tif')
odata=raster.ReadAsArray()
array0=odata[0]
#TODO 储存为图片
plt.imsave(time_point+name_list[0]+'[0]'+'.png',array0)
#TODO 转化为灰度图img
img=cv2.imread(time_point+name_list[0]+'[0]'+'.png',0)
#TODO 储存灰度图
cv2.imwrite(time_point+name_list[0]+'[0]'+'_gray'+'.png',img)
kps=fast.detect(img,None)
img_key=cv2.drawKeypoints(img,kps,img)
cv2.imwrite(time_point+name_list[0]+'[0]'+'_key'+'.png',img_key)
kps1,des1=sift.compute(img,kps)
des1=des1.astype(np.uint8)

In [5]:
raster=gdal.Open(com_name+time_point+name_list[1]+'.tif')
odata=raster.ReadAsArray()
array0=odata[0]
#TODO 储存为图片
plt.imsave(time_point+name_list[1]+'[0]'+'.png',array0)
#TODO 转化为灰度图img
img=cv2.imread(time_point+name_list[1]+'[0]'+'.png',0)
#TODO 储存灰度图
cv2.imwrite(time_point+name_list[1]+'[0]'+'_gray'+'.png',img)
kps=fast.detect(img,None)
img_key=cv2.drawKeypoints(img,kps,img)
cv2.imwrite(time_point+name_list[1]+'[0]'+'_key'+'.png',img_key)
kps2,des2=sift.compute(img,kps)
des2=des2.astype(np.uint8)

In [6]:
kps1[0].pt

(148.0, 3.0)

In [7]:
type(des1)

numpy.ndarray

In [8]:
des1.shape,des2.shape

((2230, 128), (2184, 128))

In [9]:
matches=bf.match(des1,des2)
matches=sorted(matches,key=lambda x:x.distance)
type(matches)

list

In [10]:
desc1=np.array([d/np.linalg.norm(d) for d in des1])
desc2=np.array([d/np.linalg.norm(d) for d in des2])

In [11]:
#TODO np.linalg.norm为线性代数求范数，默认为2范数
#TODO desc1实际是array，结果发现desc1经过列表循环再转换array，行列保持不变
desc1.shape,desc2.shape

((2230, 128), (2184, 128))

In [12]:
#TODO np.dot()若参数有矩阵则进行矩阵乘法
dotprods=np.dot(desc1[0,:],desc2.T)
#TODO desc1[i,:]为1x128,desc2.T为128x2184，二者矩阵乘法得1x2184
dotprods.shape

(2184,)

In [13]:
dotprods

array([0.63249535, 0.62758799, 0.63345909, ..., 0.14760153, 0.16787075,
       0.24655688])

In [14]:
#dotprods=0.9999*dotprods
#TODO np.arccos()
index=np.argsort(np.arccos(dotprods))
np.arccos(dotprods)

array([0.88602572, 0.89234509, 0.88478085, ..., 1.42265353, 1.40212696,
       1.32167048])

In [15]:
index

array([   9, 1662, 1672, ..., 2172, 2177, 2176], dtype=int64)

In [16]:
np.arccos(dotprods)[index[9]]

0.7988677313120247

In [17]:
matchscores=np.zeros((desc1.shape[0]),'int')
for j in range(desc1.shape[0]):
    dotprods=np.dot(desc1[j,:],desc2.T)
    index_angles=np.argsort(np.arccos(dotprods))
    if np.arccos(dotprods)[index_angles[0]]<0.95*np.arccos(dotprods)[index[1]]:
        matchscores[j]=index_angles[0]
matchscores

array([   9,    9,    9, ..., 2156, 2183, 2183])

In [18]:
matchscores.shape

(2230,)

In [19]:
dotprods.max()

0.9297108065224793

In [20]:
np.arccos(dotprods)

array([1.41511058, 1.30229342, 1.38626653, ..., 1.07577228, 1.11698363,
       0.37716949])

In [21]:
matchscores.nonzero()

(array([   0,    1,    2, ..., 2227, 2228, 2229], dtype=int64),)

In [22]:
matchscores.nonzero()[0].shape

(2229,)

In [23]:
np.array([1,0,2,3,5,0]).reshape(2,3)

array([[1, 0, 2],
       [3, 5, 0]])

In [24]:
np.array([1,0,2,3,5,0]).reshape(2,3).nonzero()

(array([0, 0, 1, 1], dtype=int64), array([0, 2, 0, 1], dtype=int64))

In [27]:
good_match=np.array([1,0,2,3,5,0])
good_match[good_match>0]

array([1, 2, 3, 5])